In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, max_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Bidirectional, Input
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from haversine import haversine, Unit
import ast # To parse the coordinates string
import warnings

warnings.filterwarnings('ignore')

# --- 1. CONFIGURATION ---
CSV_FILE_PATH = '../data/raw/lithuanian_weather_custom_stations.csv'

# Specify station codes. One will be the target, others will be inputs.
TARGET_STATION = 'kauno-ams'
INPUT_STATIONS = ['vilniaus-ams', 'klaipedos-ams']
ALL_STATIONS = [TARGET_STATION] + INPUT_STATIONS

# Features to be used for modeling
# CLEANUP: 'windDirection' removed as it is now handled by cyclical features
NUMERICAL_FEATURES = [
    'airTemperature', 'windSpeed', 'windGust', 'seaLevelPressure', 
    'relativeHumidity', 'precipitation', 'cloudCover'
]
CATEGORICAL_FEATURES = ['conditionCode']
TARGET_VARIABLE = 'airTemperature'


# --- 2. DATA LOADING AND FILTERING ---
print("Step 1: Loading and filtering data...")

try:
    df_raw = pd.read_csv(CSV_FILE_PATH)
except FileNotFoundError:
    print(f"ERROR: File '{CSV_FILE_PATH}' not found.")
    exit()

# Convert time and set as index
df_raw['observationTimeUtc'] = pd.to_datetime(df_raw['observationTimeUtc'])
df_raw.set_index('observationTimeUtc', inplace=True)

# IMPORTANT: Filter data to keep only December for each year
df_december = df_raw[df_raw.index.month == 12].copy()
print(f"Kept {len(df_december)} records for December only.")

# Filter data for the required stations only
df = df_december[df_december['station_code'].isin(ALL_STATIONS)].copy()

# Handle missing values
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

# Handle coordinates
def parse_coords(coord_str):
    try:
        return ast.literal_eval(coord_str)
    except (ValueError, SyntaxError):
        return {'latitude': np.nan, 'longitude': np.nan}
df['coordinates'] = df['coordinates'].apply(parse_coords)
coords = {station: df[df['station_code'] == station]['coordinates'].iloc[0] for station in ALL_STATIONS}

# --- 3. FEATURE ENGINEERING ---
print("\nStep 2: Feature Engineering...")
# Handle wind direction
wind_dir_rad = df['windDirection'] * np.pi / 180
df['wind_sin'] = np.sin(wind_dir_rad)
df['wind_cos'] = np.cos(wind_dir_rad)
ALL_FEATURES = NUMERICAL_FEATURES + ['wind_sin', 'wind_cos']

# One-Hot Encoding
df = pd.get_dummies(df, columns=CATEGORICAL_FEATURES, prefix='condition')
encoded_condition_cols = [col for col in df.columns if 'condition_' in col]
ALL_FEATURES = ALL_FEATURES + encoded_condition_cols

# --- 4. CREATING A PIVOT TABLE ---
print("\nStep 3: Creating a pivot table...")
pivot_df = df.pivot_table(index=df.index, columns='station_code', values=ALL_FEATURES)
pivot_df.interpolate(method='time', inplace=True)
pivot_df.fillna(method='ffill', inplace=True)
pivot_df.fillna(method='bfill', inplace=True)
pivot_df.columns = [f"{col[1]}_{col[0]}" for col in pivot_df.columns]
pivot_df.sort_index(inplace=True)

# Adding temporal features and distances
df_index = pivot_df.index
pivot_df['hour_sin'] = np.sin(2 * np.pi * df_index.hour / 24)
pivot_df['hour_cos'] = np.cos(2 * np.pi * df_index.hour / 24)
for station in INPUT_STATIONS:
    distance = haversine((coords[TARGET_STATION]['latitude'], coords[TARGET_STATION]['longitude']), (coords[station]['latitude'], coords[station]['longitude']), unit=Unit.KILOMETERS)
    pivot_df[f'dist_{TARGET_STATION}_to_{station}'] = distance

# Autocorrelation (ACF/PACF) plots by year
years = pivot_df.index.year.unique()
for year in years:
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    fig.suptitle(f'Autocorrelation Analysis for {year}', fontsize=16)
    year_data = pivot_df[pivot_df.index.year == year][target_temp_col].dropna()
    plot_acf(year_data, ax=axes[0], lags=48)
    plot_pacf(year_data, ax=axes[1], lags=48)
    plt.savefig(f'acf_pacf_{year}.png')
    plt.show()

# --- 6. HYBRID MODEL: SARIMAX ---
print("\nStep 5: Modeling linear patterns with SARIMAX...")
exog_features_for_sarimax = ['airTemperature', 'windSpeed', 'seaLevelPressure', 'wind_sin', 'wind_cos']
exog_cols = [col for col in pivot_df.columns if any(st in col for st in INPUT_STATIONS) and any(feat in col for feat in exog_features_for_sarimax)]
exog_data = pivot_df[exog_cols]
print(f"Using {len(exog_cols)} exogenous features for SARIMAX.")
sarimax_model = SARIMAX(pivot_df[target_temp_col], exog=exog_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24))
sarimax_results = sarimax_model.fit(disp=False)
pivot_df['sarimax_pred'] = sarimax_results.predict(start=pivot_df.index[0], end=pivot_df.index[-1], exog=exog_data)
pivot_df['residuals'] = pivot_df[target_temp_col] - pivot_df['sarimax_pred']
print("SARIMAX model trained.")

# ADDED: Visualization of SARIMAX performance by year
for year in years:
    plt.figure(figsize=(15, 6))
    plt.title(f'SARIMAX Predictions and Residuals for {year}')
    
    year_data = pivot_df[pivot_df.index.year == year]
    
    year_data[target_temp_col].plot(label='Actual Temperature', alpha=0.7)
    year_data['sarimax_pred'].plot(label='SARIMAX Forecast', linestyle='--')
    year_data['residuals'].plot(label='Residuals (Error)', color='green', alpha=0.5)
    
    plt.legend()
    plt.grid(True)
    plt.savefig(f'sarimax_performance_{year}.png')
    plt.show()

# --- 7. DATA PREPARATION AND LSTM TRAINING ---
print("\nStep 6: Preparing data and training LSTM on residuals...")
lstm_input_cols = [col for col in pivot_df.columns if col not in ['sarimax_pred', 'residuals']]
lstm_target_col = 'residuals'
input_feature_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_input_features = input_feature_scaler.fit_transform(pivot_df[lstm_input_cols])
residual_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_residuals = residual_scaler.fit_transform(pivot_df[[lstm_target_col]])

def create_hybrid_dataset(input_data, target_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(input_data) - look_back):
        dataX.append(input_data[i:(i + look_back), :])
        dataY.append(target_data[i + look_back])
    return np.array(dataX), np.array(dataY)

look_back = 72 # 3 days
X, y = create_hybrid_dataset(scaled_input_features, scaled_residuals, look_back)
train_size = int(len(X) * 0.9)
trainX, testX = X[0:train_size], X[train_size:len(X)]
trainY, testY = y[0:train_size], y[train_size:len(y)]

input_layer = Input(shape=(X.shape[1], X.shape[2]))
conv1d_layer = Conv1D(filters=64, kernel_size=5, activation='relu')(input_layer)
pool_layer = MaxPooling1D(pool_size=2)(conv1d_layer)
bilstm_layer = Bidirectional(LSTM(128, return_sequences=True))(pool_layer)
bilstm_layer = Dropout(0.3)(bilstm_layer)
bilstm_layer_2 = Bidirectional(LSTM(64, return_sequences=False))(bilstm_layer)
bilstm_layer_2 = Dropout(0.3)(bilstm_layer_2)
dense_layer = Dense(50, activation='relu')(bilstm_layer_2)
output_layer = Dense(1)(dense_layer)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
stop_early = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
history = model.fit(trainX, trainY, epochs=50, batch_size=64, validation_data=(testX, testY), callbacks=[stop_early])
print("LSTM model trained.")

# --- 8. MODEL BACKTESTING ---
print(f"\nStep 7: Testing the model on data for 2023-12-15...")
BACKTEST_DATE = '2023-12-15'
try:
    backtest_start_dt = pd.to_datetime(BACKTEST_DATE)
    if not any(pivot_df.index.date == backtest_start_dt.date()):
         raise ValueError(f"Date {BACKTEST_DATE} is missing from the dataset.")

    final_predictions = []
    predicted_residuals = []
    scaled_backtest_inputs = input_feature_scaler.transform(pivot_df[lstm_input_cols])
    for hour in range(24):
        current_dt = backtest_start_dt + pd.Timedelta(hours=hour)
        input_end_idx = pivot_df.index.get_indexer([current_dt], method='ffill')[0]
        input_start_idx = input_end_idx - look_back
        input_sequence = scaled_backtest_inputs[input_start_idx:input_end_idx].reshape(1, look_back, len(lstm_input_cols))
        predicted_scaled_residual = model.predict(input_sequence, verbose=0)
        predicted_residual = residual_scaler.inverse_transform(predicted_scaled_residual)[0][0]
        sarimax_prediction = pivot_df.loc[current_dt, 'sarimax_pred']
        final_prediction = sarimax_prediction + predicted_residual
        final_predictions.append(final_prediction)
        predicted_residuals.append(predicted_residual)

    results_df = pd.DataFrame(index=pd.date_range(start=backtest_start_dt, periods=24, freq='h'))
    results_df['Actual_Temperature'] = pivot_df.loc[BACKTEST_DATE][target_temp_col].values
    results_df['SARIMAX_Forecast'] = pivot_df.loc[BACKTEST_DATE]['sarimax_pred'].values
    results_df['LSTM_Residual_Forecast'] = predicted_residuals
    results_df['Hybrid_Forecast'] = final_predictions
    results_df.dropna(inplace=True)

    y_true = results_df['Actual_Temperature']
    y_pred = results_df['Hybrid_Forecast']
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.inf
    max_err = max_error(y_true, y_pred)
    
    metrics_data = {
        'Metric': ['RMSE', 'MAE', 'R-squared', 'MAPE (%)', 'Max Error'],
        'Value': [rmse, mae, r2, mape, max_err]
    }
    metrics_df = pd.DataFrame(metrics_data).round(4)
    
    print("\n--- Quantitative Results ---")
    print(metrics_df)
    
    print("\n--- Comparison Table ---")
    print(results_df.round(2))
    
    print("\n--- LaTeX Output for Metrics ---")
    print(metrics_df.to_latex(index=False))
    
    print("\n--- LaTeX Output for Comparison Table ---")
    print(results_df.round(2).to_latex(index=True))
    
    plt.figure(figsize=(15, 7))
    plt.title(f'Test: Forecast vs Actual Data for {TARGET_STATION} on {BACKTEST_DATE}')
    plt.plot(results_df.index, results_df['Actual_Temperature'], label='Actual Temperature', color='blue', marker='.')
    plt.plot(results_df.index, results_df['Hybrid_Forecast'], label='Hybrid Forecast', color='red', linestyle='--')
    plt.plot(results_df.index, results_df['SARIMAX_Forecast'], label='SARIMAX Forecast', color='green', linestyle=':', alpha=0.7)
    plt.xlabel('Date and Time')
    plt.ylabel('Temperature, °C')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('backtest_forecast.png')
    plt.show()

    heatmap_data = results_df[['Actual_Temperature', 'Hybrid_Forecast']].copy()
    heatmap_data.index = heatmap_data.index.strftime('%H:%M')
    plt.figure(figsize=(8, 10))
    sns.heatmap(heatmap_data.T, annot=True, fmt=".1f", cmap="coolwarm", linewidths=.5)
    plt.title(f'Heatmap of Actual vs. Forecast Temperatures on {BACKTEST_DATE}')
    plt.xlabel('Hour of Day')
    plt.ylabel('')
    plt.savefig('backtest_heatmap.png')
    plt.show()

except (KeyError, ValueError, IndexError) as e:
    print(f"ERROR during backtesting: {e}")

# --- 9. FINAL FORECAST FOR 2025-12-31 ---
print("\nStep 8: Final forecast for 2025-12-31...")
future_predictions = []
last_known_input_sequence = scaled_input_features[-look_back:].reshape(1, look_back, len(lstm_input_cols))
last_known_exog = exog_data.iloc[-1:].values.reshape(1, -1)

sarimax_future_forecast = sarimax_results.get_forecast(steps=24, exog=np.repeat(last_known_exog, 24, axis=0))
sarimax_future_preds = sarimax_future_forecast.predicted_mean

current_sequence = last_known_input_sequence
for i in range(24):
    predicted_scaled_residual = model.predict(current_sequence, verbose=0)
    predicted_residual = residual_scaler.inverse_transform(predicted_scaled_residual)[0][0]
    
    final_prediction = sarimax_future_preds.iloc[i] + predicted_residual
    future_predictions.append(final_prediction)
    
    new_step = current_sequence[0, -1, :].copy() 
    current_sequence = np.append(current_sequence[:, 1:, :], [[new_step]], axis=1)


forecast_df = pd.DataFrame(future_predictions, columns=['Forecast_Temperature'])
forecast_df.index = pd.date_range(start='2025-12-31 00:00:00', periods=24, freq='h')

print("Hourly forecast for 2025-12-31:")
print(forecast_df.round(2))

plt.figure(figsize=(15, 7))
plt.title(f'Final Forecast for {TARGET_STATION} on 2025-12-31')
plt.plot(forecast_df.index, forecast_df['Forecast_Temperature'], label='Forecasted Temperature', color='red', marker='o')
plt.xlabel('Date and Time')
plt.ylabel('Temperature, °C')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('final_forecast_2025.png')
plt.show()



ImportError: Traceback (most recent call last):
  File "C:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.